In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mineralML as mm


In [19]:
# %pip install mineralML

In [20]:
def process_excel_file(file_path, sheet_name=1):
    # Load the Excel file from the specified sheet without headers
    data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Find indices where 'Element' is found in column 0 to determine the start of data blocks
    data_blocks_indices = data.index[data[0] == 'Element'].tolist()
    
    # Initialize an empty DataFrame to store processed data
    processed_data = []

    # Iterate through each data block
    for i, start in enumerate(data_blocks_indices):
        # The label for the block is in the row just before the 'Element' row
        label = data.iloc[start - 1, 0]
        
        # Identify the end of the block by finding the next 'Total' row after 'Element'
        total_index = data.index[(data[0] == 'Total') & (data.index > start)].min()
        end = total_index  # Set end to the row before 'Total
        
        # Extract data block for current sample
        block = data.iloc[start + 1:end]  # Extract rows between 'Element' and 'Total'
        headers = ['Element'] + data.iloc[start, 1:].dropna().values.tolist()  # Get headers from the 'Element' row
        
        # Ensure the number of columns matches expected headers
        block.columns = headers[:block.shape[1]]
        
        # Prepare a row dictionary to append to the DataFrame
        row_dict = {'Sample Name': label}
        for _, row in block.iterrows():
            element = row['Element']
            
            for col in row.index[1:]:  # Skip the 'Element' column for key names
                if pd.notna(row[col]):  # Check for NaN
                    col_name = f"{col}_{element}"  # Use header from 'Element' row for column names
                    row_dict[col_name] = row[col]

        # Capture 'Total_wt%' and 'Total_Oxide %' from the 'Total' row
        total_row = data.iloc[total_index, :]  # Correctly access the 'Total' row
        row_dict['Total_wt%'] = total_row.get(headers.index('Wt%'), None)  # Use dynamic index for 'Wt%'
        row_dict['Total_Oxide%'] = total_row.get(headers.index('Oxide %'), None)  # Use dynamic index for 'Oxide %'
        row_dict['Cation_Sum'] = total_row.get(headers.index('Number of Ions'), None)  # Use dynamic index for 'Oxide %'
        row_dict['Cation_Sum']=row_dict['Cation_Sum'].replace('(Cation sum)', '')

                    
        processed_data.append(row_dict)

    return pd.DataFrame(processed_data)



In [21]:
def sort_columns(df, norm = False):
    # Start with fixed priority columns
    priority_cols = ['Sample Name', 'Total_wt%', 'Total_Oxide%', 'Cation_Sum']

    # Define exact matches for metrics and their intended order
    col_order = ['Oxide %_', 'Oxide % Sigma_', 'norm_Oxide %_', 'Wt%_', 'Wt% Sigma_']


    element_to_oxide = {
        'Si': 'SiO2', 'Ti': 'TiO2', 'Al': 'Al2O3', 'Ca': 'CaO',
        'Cr': 'Cr2O3', 'Fe': 'FeO', 'Ni': 'NiO', 'Na': 'Na2O',
        'Mg': 'MgO','Mn': 'MnO',
        'K': 'K2O', 'P': 'P2O5'
    }

    # Rename columns based on the mapping
    new_columns = {}
    for col in df.columns:
        if 'Oxide' in col:
            for element, oxide in element_to_oxide.items():
                if f'_{element}' in col:
                    new_col_name = col.replace(f'_{element}', f'_{oxide}')
                    new_columns[col] = new_col_name

    df.rename(columns=new_columns, inplace=True)
    
    
    # Find the columns that are going to be involved in the normalization
    desired_string = 'Oxide %_'
    matching_columns = [col for col in df.columns if desired_string in col]
    matching_column_indices = [df.columns.get_loc(col) for col in matching_columns]
        
    # Convert SO3 to SO2
    if 'Oxide %_S' in list(df.keys()):
        df = df.rename(columns={'Oxide %_S': 'Oxide %_SO2'})
        df['Oxide %_SO2'] = df['Oxide %_SO2']*(32.065+2*15.999)/(32.065+3*15.999)

    # normalization
    df_norm = df.iloc[:, matching_column_indices]
    df_norm = df_norm.map(lambda x: 0 if x < 0 else x)
    sum = df_norm.sum(axis = 1)
    df_norm = 100*df_norm.div(sum, axis = 0)

    df_norm = df_norm.add_suffix('_norm')
    df_norm = df_norm.add_prefix('norm_')
    df_norm = df_norm.fillna(0.0)

    df = pd.concat([df, df_norm], axis=1)

    # if norm is true, switch normalized and un-normalized analyses
    if norm is True:
        elements = set([col.split('_')[-1].split('_norm')[0] for col in df.columns if 'Oxide %_' in col])
        
        # Swap the data between corresponding columns
        for element in elements:
            if element != 'norm':
                oxide_col = f'Oxide %_{element}'
                norm_oxide_col = f'norm_Oxide %_{element}_norm'
                
                # Swap the data using a temporary variable
                temp = df[oxide_col].copy()
                df[oxide_col] = df[norm_oxide_col]
                df[norm_oxide_col] = temp


    # Collect and sort metric-specific columns using more accurate matching
    metric_cols = []
    for metric in col_order:
        # This loop will check if the columns end with the metric name preceded by an underscore
        # to ensure we match 'Oxide %_Mg' but not 'Something Oxide %_Mg'
        metric_cols.extend(sorted([col for col in df.columns if col.startswith(metric)]))

    
    # Collect all other columns not in priority or metric-specific lists
    other_cols = [col for col in df.columns if col not in priority_cols and col not in metric_cols]

    # Move 'Apparent Concentration' columns to the end
    apparent_concentration_cols = [col for col in other_cols if 'Apparent Concentration' in col]
    other_cols = [col for col in other_cols if col not in apparent_concentration_cols]

    # Combine all columns in the desired order
    final_cols = priority_cols + metric_cols + sorted(other_cols) + sorted(apparent_concentration_cols)

    # Debugging: Print out final column order
    # Move 'Apparent Concentration' columns to the end
    for bad_names in ['Apparent Concentration', 'Factory Standard', 'Line', 'Number of Ions']:
        
        apparent_concentration_cols = [col for col in df.columns if bad_names in col]
        final_cols = [col for col in final_cols if col not in apparent_concentration_cols] + apparent_concentration_cols

    df_out=df[final_cols]

    # replace column headers to make compatible with MinML
    
    df_out.rename(columns=lambda x: x.replace('norm_Oxide %_', ''), inplace=True)   
    if norm is True:
        df_out.rename(columns=lambda x: x.replace('_norm','_unnorm'), inplace=True)
         
    df_out.rename(columns=lambda x: x.replace('Oxide %_', ''), inplace=True)
    df_out.rename(columns=lambda x: x.replace('Number of Ions', '#_ions_'), inplace=True)
    df_out.rename(columns=lambda x: x.replace('FeO', 'FeOt'), inplace=True)

    # Reorder DataFrame
    return df_out


In [22]:
# do MinML classification - assign cation totals and split into separate sheets in output excel.
def minClass(df, oxygen_num = 1):
    # prep dataframe for MinML analysis
    
    df['Mineral']='Olivine'
    df['entity_id']=df.index
    
    df_pred_nn, probability_matrix = mm.predict_class_prob_nn(df)

    # Extract key columns into main results dataframe
    # df['Predict_Mineral'] = df_pred_nn['Predict_Mineral']
    # df['Predict_Probabilty'] = df_pred_nn['Predict_Probability']
    columns = df.columns.tolist()
    if 'Predict_Mineral' in columns:
        columns.remove('Predict_Mineral')
    if 'Predict_Probability' in columns:
        columns.remove('Predict_Probability')
    columns.insert(1, 'Predict_Mineral')
    columns.insert(2, 'Predict_Probability')

    df = df.reindex(columns = columns)

    Oxygen_numbers = {'Amphibole': 22, 'Apatite': 4, 'Biotite': 10, 'Clinopyroxene': 6,
                      'Garnet': 12, 'Ilmenite': 3, 'K-Feldspar': 8, 'Magnetite': 4,
                      'Muscovite': 10, 'Olivine': 4, 'Orthopyroxene': 6, 'Plagioclase': 8,
                      'Quartz': 2, 'Rutile': 2, 'Spinel': 4, 'Tourmaline': 27, 'Zircon': 4}

    for i in df['Predict_Mineral'].unique():
        df.loc[df['Predict_Mineral'] == i, 'Cation_Sum'] = Oxygen_numbers[i]*df.loc[df['Predict_Mineral'] == i, 'Cation_Sum'].astype(float)/oxygen_num

    return df

    

In [23]:
result = process_excel_file('MountR3_EDS_5-8-24_mod.xlsx', sheet_name="For_transform")
sorted_df = sort_columns(result)
sorted_df['K2O']=0 # Sarah will add a fix
sorted_df['Na2O']=0
minml_df = minClass(sorted_df)

minml_df

,Sample Name,Predict_Mineral,Predict_Probability,Total_wt%,Total_Oxide%,Cation_Sum,Al2O3,CaO,Cr2O3,FeOt,...,#_ions__Ni,#_ions__Al,#_ions__Ti,#_ions__Cr,K2O,Na2O,Mineral,entity_id,Second_Predict_Mineral,Second_Predict_Probability
0,Springwater_1,Olivine,0.978915,101.29,101.29,12.04,NaN,0.02,NaN,16.56,...,0.00,NaN,NaN,NaN,0,0,Olivine,0,Orthopyroxene,0.014325
1,Springwater_2,Olivine,0.978899,101.28,101.28,12.04,NaN,0.04,NaN,16.60,...,0.00,NaN,NaN,NaN,0,0,Olivine,1,Orthopyroxene,0.014323
2,SanCarlos_1,Olivine,0.979997,99.15,99.15,12.04,NaN,0.10,NaN,9.71,...,0.01,NaN,NaN,NaN,0,0,Olivine,2,Orthopyroxene,0.013403
3,SanCarlos_2,Olivine,0.980074,99.23,99.23,12.04,NaN,0.11,NaN,9.67,...,0.01,NaN,NaN,NaN,0,0,Olivine,3,Orthopyroxene,0.013311
4,JThypersthene_1,Orthopyroxene,0.978630,98.05,98.05,24.06,0.9,1.30,0.67,15.12,...,NaN,0.04,0.0,0.02,0,0,Olivine,4,Amphibole,0.012176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,187-1-123-reg1,Olivine,0.978806,103.73,103.73,12.04,NaN,0.40,NaN,13.16,...,0.01,NaN,NaN,NaN,0,0,Olivine,78,Orthopyroxene,0.015127
79,SanCarlos_5,Olivine,0.979948,100.03,100.03,12.04,NaN,0.11,NaN,9.67,...,0.01,NaN,NaN,NaN,0,0,Olivine,79,Orthopyroxene,0.013597
80,SanCarlos_6,Olivine,0.979774,99.58,99.58,12.04,NaN,0.09,NaN,9.70,...,0.01,NaN,NaN,NaN,0,0,Olivine,80,Orthopyroxene,0.013769
81,Springwater_5,Olivine,0.979227,100.69,100.69,12.04,NaN,0.01,NaN,16.41,...,0.00,NaN,NaN,NaN,0,0,Olivine,81,Orthopyroxene,0.013835


In [24]:
xy_pos=pd.read_excel('MountR3_EDS_5-8-24_mod.xlsx', sheet_name="Metadata")
xy_pos

,Label,Stage X (µm),Stage Y (µm),Sample_Name_4annot
0,Springwater_1,25862.25,-15111.00,Springwater_1
1,Springwater_2,25882.25,-15103.75,Springwater_2
2,SanCarlos_1,20207.75,-11529.50,SanCarlos_1
3,SanCarlos_2,20220.75,-11528.25,SanCarlos_2
4,JThypersthene_1,13758.25,-13655.00,JThypersthene_1
...,...,...,...,...
78,187-1-123-reg1,20868.25,25166.75,187-1-123_reg1
79,SanCarlos_5,20207.50,-11551.00,SanCarlos_5
80,SanCarlos_6,20211.25,-11546.75,SanCarlos_6
81,Springwater_5,25875.75,-15118.25,Springwater_5


In [27]:
final_df=pd.merge(left=minml_df,right=xy_pos,left_on="Sample Name",right_on="Label")
final_df
final_df.to_excel('MountR3_EDS_5-8-24_processed.xlsx')